In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error\

import tensorflow as tf
from tensorflow.keras import Sequential, layers
from tensorflow import keras

import_path = Path("/kaggle/input/tabular-playground-series-jan-2021/")

In [ ]:
train = pd.read_csv(import_path / 'train.csv', index_col = 'id')
test = pd.read_csv(import_path / 'test.csv', index_col = 'id')
submission = pd.read_csv(import_path / 'sample_submission.csv', index_col = 'id')

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
plt.figure(figsize=(30,15))
for i, col in enumerate(train.columns):
    plt.subplot(3,5,(i+1))
    sns.distplot(train[col])

In [ ]:
train.info()

In [ ]:
target = train.pop('target')

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train = pd.DataFrame(scaler.fit_transform(train),columns=train.columns)
test = pd.DataFrame(scaler.transform(test),columns=test.columns)

In [ ]:
train.iloc[train_index,:]

In [ ]:
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping

cv = KFold(n_splits=5, shuffle=True, random_state=2020)

y_preds = []
models = []
oof_train = np.zeros((len(train),))

for fold_id, (train_index, valid_index) in enumerate(cv.split(train, target)):
    X_tr = train.iloc[train_index,:]
    X_val = train.iloc[valid_index, :]
    y_tr = target.iloc[train_index]
    y_val = target.iloc[valid_index]
    
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=[14]),
        layers.Dropout(0.3),
        layers.BatchNormalization(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.BatchNormalization(),
        layers.Dense(1, activation='linear')
    ])
    
    model.compile(optimizer='adam',
                 loss='mse',
                 metrics=[keras.metrics.RootMeanSquaredError()]
    )
    
    early_stopping = EarlyStopping(
        min_delta=0.001,
        patience=10, 
        restore_best_weights=True,
    )
    
    model.fit(
        X_tr, y_tr,
        validation_data=(X_val, y_val),
        batch_size=30000,
        epochs=1000,
        callbacks=[early_stopping]
    )
    
    oof_train[valid_index] = model.predict(X_val).reshape(1,-1)[0]
    y_pred = model.predict(test).reshape(1,-1)[0]
    
    y_preds.append(y_pred)
    models.append(model)

In [ ]:
mean_squared_error(target, oof_train, squared=False)

In [ ]:
submission['target'] = sum(y_preds) / len(y_preds)
submission.to_csv('tensorflow_kfold.csv')

In [ ]:
y_preds

In [ ]:
# Show the learning curves

# history_df = pd.DataFrame(history.history)
# history_df.loc[:, ['loss']].plot()
# # used validation
# # history_df.loc[:, ['loss', 'val_loss']].plot()

In [ ]:
# submission['target'] = model.predict(test) 
# submission.to_csv('tensorflow_baseline.csv')